In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import cv2 as cv 
import glob
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler,Subset
from torchvision import transforms
from IPython.display import HTML
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torchvision  import datasets 
from sklearn.model_selection import KFold
import optuna
from optuna.trial import Trial
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Subset, DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
combined_data = "/kaggle/input/deepfake-and-real-images/Dataset/Test"

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),         # Convert to Tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize
])

In [5]:
combinedData = datasets.ImageFolder(combined_data, transform=transform)

In [6]:
combinedData

Dataset ImageFolder
    Number of datapoints: 10905
    Root location: /kaggle/input/deepfake-and-real-images/Dataset/Test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [7]:
kf = KFold(n_splits=3, shuffle=True, random_state=42) 

In [8]:
samples = len(combinedData)
samples

10905

In [9]:
folds = list(kf.split(np.arange(samples)))

In [10]:
folds

[(array([    1,     2,     4, ..., 10900, 10901, 10902]),
  array([    0,     3,     8, ..., 10899, 10903, 10904])),
 (array([    0,     1,     3, ..., 10902, 10903, 10904]),
  array([    2,    15,    18, ..., 10889, 10900, 10901])),
 (array([    0,     2,     3, ..., 10901, 10903, 10904]),
  array([    1,     4,     5, ..., 10896, 10897, 10902]))]

# Split our data into 3 Kfolds

In [11]:
for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
    print(f"Fold {fold_idx + 1}")
    
    train_size = int(0.75* len(train_val_idx)) 
    val_size = len(train_val_idx) - train_size #25
    
    train_idx = train_val_idx[:train_size]
    val_idx = train_val_idx[train_size:]
    
    # Create subsets
    train_set = Subset(combinedData, train_idx)
    val_set = Subset(combinedData, val_idx)
    test_set = Subset(combinedData, test_idx)
    
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
    
    print(f"Train size: {len(train_set)}")
    print(f"Validation size: {len(val_set)}")
    print(f"Test size: {len(test_set)}")

Fold 1
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 2
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 3
Train size: 5452
Validation size: 1818
Test size: 3635


# Define our model ResNet-50

In [12]:

def get_model(name):
    if name=="resnet-50":
        model= models.resnet50(pretrained=True)
        for param in model.parameters():
            param.requires_grad = False
    
        model.fc = nn.Linear(model.fc.in_features, 2)
    else:
        raise ValueError("Model name must be 'alexnet' or 'vgg16'")
    return model

In [ ]:
# Fine-tune


# def get_model(name):
#     if name == "resnet-50":
#         model = models.resnet50(pretrained=True)
        
#         # Freeze all layers initially
#         for param in model.parameters():
#             param.requires_grad = False
            
#         # Unfreeze the final two residual blocks (layer3 and layer4)
#         for layer in [model.layer3, model.layer4]:
#             for param in layer.parameters():
#                 param.requires_grad = True
                
#         # Unfreeze and modify the final fully connected layer
#         model.fc = nn.Linear(model.fc.in_features, 2)
        
#         # Print layer status
#         def count_trainable_params(model):
#             return sum(p.numel() for p in model.parameters() if p.requires_grad)
            
#         print(f"Total trainable parameters: {count_trainable_params(model):,}")
        
#     else:
#         raise ValueError("Model name must be 'resnet-50'")
    
#     return model

# Training function

In [13]:
def TrainingModels(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)  #loss function used to compute the error between predictions and true labels
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1) # max score from the scores of two classes
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc




In [55]:
# Fine-tune

# def TrainingModels(model, train_loader, criterion, optimizer):
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0
    
#     # Add gradient clipping to prevent exploding gradients
#     max_grad_norm = 1.0
    
#     for images, labels in tqdm(train_loader, desc="Training", leave=False):
#         images, labels = images.to(device), labels.to(device)
        
#         outputs = model(images)
#         loss = criterion(outputs, labels)
        
#         optimizer.zero_grad()
#         loss.backward()
        
#         # Clip gradients
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
#         optimizer.step()

#         running_loss += loss.item() * images.size(0)
#         _, preds = torch.max(outputs, 1)
#         correct += (preds == labels).sum().item()
#         total += labels.size(0)

#     epoch_loss = running_loss / total
#     epoch_acc = correct / total
#     return epoch_loss, epoch_acc

# Eval function

In [14]:
def evaluate(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Optimize the Learning rate using Optuna

In [15]:
def LR_optimization(trial: Trial):
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    
    # Iterate over folds
    fold_results = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"\nFold {fold_idx + 1}")
        
        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

        
        for model_name in ["resnet-50"]:
            print(f"\nTraining model: {model_name.upper()} on Fold {fold_idx + 1}")
            
            # Initialize model
            model = get_model(model_name).to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            for epoch in range(5):
                current_lr = optimizer.param_groups[0]["lr"]
                print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
                # Train and evaluate
                train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
                val_loss, val_acc = evaluate(model, val_loader, criterion)

                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
                print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            _, val_acc = evaluate(model, val_loader, criterion)
            fold_results.append(val_acc) 

    mean_val_acc = sum(fold_results) / len(fold_results)
    return mean_val_acc



In [ ]:
# Fine-tune

# def LR_optimization(trial: Trial):
#     # Use a smaller learning rate range for fine-tuning
#     lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
    
#     # Rest of the function remains the same
#     fold_results = []
#     for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
#         print(f"\nFold {fold_idx + 1}")
        
#         train_size = int(0.75 * len(train_val_idx))
#         train_idx = train_val_idx[:train_size]
#         val_idx = train_val_idx[train_size:]

#         train_set = Subset(combinedData, train_idx)
#         val_set = Subset(combinedData, val_idx)
#         test_set = Subset(combinedData, test_idx)

#         train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
#         val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
#         test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

#         for model_name in ["resnet-50"]:
#             print(f"\nFine-tuning model: {model_name.upper()} on Fold {fold_idx + 1}")
            
#             model = get_model(model_name).to(device)
#             criterion = nn.CrossEntropyLoss()
            
#             # Use a smaller weight decay for fine-tuning
#             optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            
#             # Add learning rate scheduler for fine-tuning
#             scheduler = optim.lr_scheduler.ReduceLROnPlateau(
#                 optimizer, mode='max', factor=0.5, patience=2, verbose=True
#             )

#             for epoch in range(5):
#                 current_lr = optimizer.param_groups[0]["lr"]
#                 print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
#                 train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
#                 val_loss, val_acc = evaluate(model, val_loader, criterion)
                
#                 # Update learning rate based on validation accuracy
#                 scheduler.step(val_acc)

#                 print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
#                 print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
#             _, val_acc = evaluate(model, val_loader, criterion)
#             fold_results.append(val_acc)

#     mean_val_acc = sum(fold_results) / len(fold_results)
#     return mean_val_acc

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(LR_optimization, n_trials=5)

best_trial = study.best_trial
print("\nBest Trial:")
print(f"  Value (Mean Validation Accuracy): {best_trial.value}")
print(f"  Params: {best_trial.params}")

[I 2024-12-24 21:53:13,775] A new study created in memory with name: no-name-f91a8fb1-0f80-4626-b5b3-a026a07f58d4
<ipython-input-15-e1e71bb4b32b>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date wei


Fold 1

Training model: RESNET-50 on Fold 1


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 171MB/s] 



Epoch 1/5 | Learning Rate: 0.000411


Train Loss: 0.5910, Train Acc: 0.6818
Val Loss: 0.7907, Val Acc: 0.4422

Epoch 2/5 | Learning Rate: 0.000411


...

Train Loss: 0.4985, Train Acc: 0.7515
Val Loss: 0.8834, Val Acc: 0.4197

Epoch 4/5 | Learning Rate: 0.000436


Train Loss: 0.4885, Train Acc: 0.7537
Val Loss: 0.7363, Val Acc: 0.5528

Epoch 5/5 | Learning Rate: 0.000436


Train Loss: 0.4864, Train Acc: 0.7544
Val Loss: 0.6578, Val Acc: 0.6342


[I 2024-12-24 22:49:07,476] Trial 4 finished with value: 0.4457279061239457 and parameters: {'lr': 0.00043648808303298333}. Best is trial 2 with value: 0.6316464979831315.



Best Trial:
  Value (Mean Validation Accuracy): 0.6316464979831315
  Params: {'lr': 0.0006172423196498955}


# After the Optuna optimization process, we will use the best trial to set the learning rate

In [18]:
# Best learning rate from Optuna
best_lr = 0.0006172423196498955

In [19]:
final_results = {}

for model_name in ["resnet-50"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    # Initialize model
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    
    # Create DataLoaders (assuming combinedData is split into train and test datasets)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    # Train the model
    for epoch in range(10):  # Example: 10 epochs
        print(f"\nEpoch {epoch + 1}/10")
        
        # Train and evaluate
        train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    
    # Final evaluation on the test set
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    
    # Store results
    final_results[model_name] = {
        "Test Loss": test_loss,
        "Test Accuracy": test_acc
    }

# Print final results
print("\nFinal Results:")
for model_name, metrics in final_results.items():
    print(f"{model_name.upper()}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")


Training model: RESNET-50 with best learning rate 0.0006172423196498955

Epoch 1/10


Train Loss: 0.5776, Train Acc: 0.7041

Epoch 2/10


Train Loss: 0.5123, Train Acc: 0.7427

Epoch 3/10


Train Loss: 0.5019, Train Acc: 0.7449

Epoch 4/10


Train Loss: 0.4985, Train Acc: 0.7504

Epoch 5/10


...

Train Loss: 0.4753, Train Acc: 0.7625

Epoch 8/10


Train Loss: 0.4749, Train Acc: 0.7639

Epoch 9/10


Train Loss: 0.4656, Train Acc: 0.7781

Epoch 10/10


Train Loss: 0.4587, Train Acc: 0.7735


Test Loss: 0.5158, Test Acc: 0.7376

Final Results:
RESNET-50:
  Test Loss: 0.5158
  Test Accuracy: 0.7376


In [20]:
# Function to evaluate the model and get performance metrics
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Initialize cross-validation metrics storage
confusion_matrices = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

best_lr = 0.0006172423196498955

for model_name in ["resnet-50"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"Fold {fold_idx + 1}")

        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
        
        
        model = get_model(model_name).to(device)
        optimizer = optim.Adam(model.parameters(), lr=best_lr)
        criterion = nn.CrossEntropyLoss()

        # Train the model
        for epoch in range(10):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f"Epoch {epoch+1}/10, Loss: {running_loss / len(train_loader):.4f}")

    
        preds, labels = evaluate_model(model, test_loader)

        
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average='weighted', zero_division=0)
        recall = recall_score(labels, preds, average='weighted', zero_division=0)
        f1 = f1_score(labels, preds, average='weighted', zero_division=0)
        cm = confusion_matrix(labels, preds)

        # Store metrics for the fold
        confusion_matrices.append(cm)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold_idx + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Confusion Matrix:\n{cm}\n")

    # Average metrics across folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1_score = np.mean(f1_scores)
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)

    # Print final average metrics for the model
    print(f"\nFinal Results for {model_name.upper()}:")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1-Score: {avg_f1_score:.4f}")
    print(f"Average Confusion Matrix:\n{avg_confusion_matrix}")


Training model: RESNET-50 with best learning rate 0.0006172423196498955
Fold 1
Epoch 1/10, Loss: 0.5909
Epoch 2/10, Loss: 0.5197
Epoch 3/10, Loss: 0.4995
Epoch 4/10, Loss: 0.4874
Epoch 5/10, Loss: 0.4794
Epoch 6/10, Loss: 0.4684
Epoch 7/10, Loss: 0.4716
Epoch 8/10, Loss: 0.4683
Epoch 9/10, Loss: 0.4607
Epoch 10/10, Loss: 0.4483
Fold 1 - Accuracy: 0.7243, Precision: 0.7252, Recall: 0.7243, F1-Score: 0.7236
Confusion Matrix:
[[1436  428]
 [ 574 1197]]

Fold 2


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.5787
Epoch 2/10, Loss: 0.5188
Epoch 3/10, Loss: 0.5018
Epoch 4/10, Loss: 0.4925
Epoch 5/10, Loss: 0.4815
Epoch 6/10, Loss: 0.4688
Epoch 7/10, Loss: 0.4737
Epoch 8/10, Loss: 0.4735
Epoch 9/10, Loss: 0.4599
Epoch 10/10, Loss: 0.4647
Fold 2 - Accuracy: 0.7175, Precision: 0.7313, Recall: 0.7175, F1-Score: 0.7127
Confusion Matrix:
[[1547  287]
 [ 740 1061]]

Fold 3


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.5683
Epoch 2/10, Loss: 0.5226
Epoch 3/10, Loss: 0.4967
Epoch 4/10, Loss: 0.4887
Epoch 5/10, Loss: 0.4827
Epoch 6/10, Loss: 0.4797
Epoch 7/10, Loss: 0.4731
Epoch 8/10, Loss: 0.4718
Epoch 9/10, Loss: 0.4628
Epoch 10/10, Loss: 0.4609
Fold 3 - Accuracy: 0.7260, Precision: 0.7381, Recall: 0.7260, F1-Score: 0.7231
Confusion Matrix:
[[1495  299]
 [ 697 1144]]


Final Results for RESNET-50:
Average Accuracy: 0.7226
Average Precision: 0.7315
Average Recall: 0.7226
Average F1-Score: 0.7198
Average Confusion Matrix:
[[1492.66666667  338.        ]
 [ 670.33333333 1134.        ]]
